In [13]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd

sb.set()
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("cleaned-Housing.csv")
data.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,0,3/1/2012,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45,Improved,1986,250000.0
1,1,3/1/2012,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44,Improved,1980,265000.0
2,2,3/1/2012,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68,New Generation,1980,315000.0
3,3,3/1/2012,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67,New Generation,1984,320000.0
4,4,3/1/2012,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67,New Generation,1980,321000.0


In [8]:
# Linear regression
# Include YEAR and LEASE_DATE into X (predictor)
data['resale_price'].fillna(0, inplace=True)
data = data.dropna(subset=['floor_area_sqm'])

X = data[['floor_area_sqm']]
y = data['resale_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linear_reg_model = LinearRegression()

# Fit the model on the training data
linear_reg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = linear_reg_model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print out the model performance metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared Score: {r2}')

Mean Squared Error (MSE): 49101189726.02468
R-squared Score: 0.009338818928007475


In [9]:
from sklearn.linear_model import Ridge

# Initialize the Ridge Regression model
# The alpha parameter controls the degree of regularization (larger values imply more regularization)
ridge_reg_model = Ridge(alpha=1.0)

# Fit the model on the training data
ridge_reg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_ridge = ridge_reg_model.predict(X_test)

# Evaluate the model performance
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Print out the model performance metrics for Ridge Regression
print('Ridge Regression:')
print(f'Mean Squared Error (MSE): {mse_ridge}')
print(f'R-squared Score: {r2_ridge}')


Ridge Regression:
Mean Squared Error (MSE): 49101189724.921715
R-squared Score: 0.009338818950260785


In [10]:
from sklearn.linear_model import BayesianRidge

# Initialize the Bayesian Ridge Regression model
bayesian_ridge_model = BayesianRidge()

# Fit the model on the training data
bayesian_ridge_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_bayesian = bayesian_ridge_model.predict(X_test)

# Evaluate the model performance
mse_bayesian = mean_squared_error(y_test, y_pred_bayesian)
r2_bayesian = r2_score(y_test, y_pred_bayesian)

# Print out the model performance metrics for Bayesian Ridge Regression
print('Bayesian Ridge Regression:')
print(f'Mean Squared Error (MSE): {mse_bayesian}')
print(f'R-squared Score: {r2_bayesian}')


Bayesian Ridge Regression:
Mean Squared Error (MSE): 49101131295.81488
R-squared Score: 0.009339997810657197


In [11]:
# For Ridge Regression
ridge_coeffs = ridge_reg_model.coef_
print(f'Ridge Regression Coefficients: {ridge_coeffs}')

# For Bayesian Ridge Regression
bayesian_ridge_coeffs = bayesian_ridge_model.coef_
print(f'Bayesian Ridge Regression Coefficients: {bayesian_ridge_coeffs}')


Ridge Regression Coefficients: [967.85115515]
Bayesian Ridge Regression Coefficients: [967.21147542]


In [15]:
df = data

# Encode categorical variables
label_encoders = {}
categorical_cols = ['flat_type', 'storey_range', 'flat_model']
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Split the data into features and target variable
X = df[['flat_type', 'storey_range', 'flat_model', 'floor_area_sqm']]
y = df['resale_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Gradient Boosting Regressor model
gb_regressor = GradientBoostingRegressor()

# Train the model
gb_regressor.fit(X_train, y_train)

# Make predictions
y_pred = gb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 44368038553.20795


In [16]:
# Define the lower and upper quantile thresholds
lower_quantile = 0.25  # 5th percentile
upper_quantile = 0.75  # 95th percentile

# Filter out the outliers based on the resale price column
lower_threshold = df['resale_price'].quantile(lower_quantile)
upper_threshold = df['resale_price'].quantile(upper_quantile)
cleaned_df = df[(df['resale_price'] >= lower_threshold) & (df['resale_price'] <= upper_threshold)]

# Split the cleaned data into features and target variable
X = cleaned_df[['flat_type', 'storey_range', 'flat_model', 'floor_area_sqm']]
y = cleaned_df['resale_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Gradient Boosting Regressor model
gb_regressor = GradientBoostingRegressor()

# Train the model
gb_regressor.fit(X_train, y_train)

# Make predictions
y_pred = gb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error after removing outliers:", mse)

Mean Squared Error after removing outliers: 5464626336.649795


In [17]:
# Heloooooo
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from datetime import datetime

# Remove rows with missing values
df.dropna(inplace=True)

# Convert lease_commence_date to datetime
df['lease_commence_date'] = pd.to_datetime(df['lease_commence_date'], format='ISO8601')

# Filter the dataset for years from 1980 to 2012
df_filtered = df[(df['lease_commence_date'] >= datetime(1980, 1, 1)) & (df['lease_commence_date'] <= datetime(2012, 12, 31))]

# Define features (X) and target variable (y)
X = df_filtered[['flat_type', 'storey_range', 'flat_model', 'floor_area_sqm']]
y = df_filtered['resale_price']

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Initialize an empty dictionary to store SVR models and MSE for each interval
models_mse = {}



# Iterate over 5-year intervals from 1980 to 2012
for year_start in range(1980, 2013, 5):
    year_end = year_start + 4
    
    # Filter the dataset for the current 5-year interval
    # Filter the dataset for years from 1980 to 2012
    df_interval = df[(df['lease_commence_date'].dt.year >= year_start) & (df['lease_commence_date'].dt.year <= year_end)]

    print(df_interval.info())
    continue
    
    # Skip the interval if there is no data
    if df_interval.empty:
        continue
    
    # Define features and target variable for the current interval
    X_interval = df_interval[['flat_type', 'storey_range', 'flat_model', 'floor_area_sqm']]
    y_interval = df_interval['resale_price']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_interval, y_interval, test_size=0.2, random_state=42)
    
    # Initialize Support Vector Regression model
    svr_regressor = SVR()
    
    # Define hyperparameters for randomized search
    param_dist = {
        'C': [0.1, 1, 10],
        'gamma': [0.01, 0.1, 1]
    }
    
    # Perform randomized search to find the best hyperparameters
    random_search = RandomizedSearchCV(svr_regressor, param_dist, cv=3, scoring='neg_mean_squared_error', n_iter=10, random_state=42)
    random_search.fit(X_train, y_train)
    
    # Get the best hyperparameters
    best_params = random_search.best_params_
    
    # Initialize SVR with best hyperparameters
    best_svr_regressor = SVR(**best_params)
    
    # Train the model
    best_svr_regressor.fit(X_train, y_train)
    
    # Make predictions
    y_pred = best_svr_regressor.predict(X_test)
    
    # Calculate Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    
    # Store the trained SVR model and MSE for the current interval
    models_mse[(year_start, year_end)] = {'model': best_svr_regressor, 'mse': mse}

# Print the MSE for each interval
for interval, data in models_mse.items():
    print(f"5-year Interval: {interval}, Mean Squared Error: {data['mse']}")

<class 'pandas.core.frame.DataFrame'>
Index: 21310 entries, 1 to 167349
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0           21310 non-null  int64         
 1   month                21310 non-null  object        
 2   town                 21310 non-null  object        
 3   flat_type            21310 non-null  int64         
 4   block                21310 non-null  object        
 5   street_name          21310 non-null  object        
 6   storey_range         21310 non-null  int64         
 7   floor_area_sqm       21310 non-null  int64         
 8   flat_model           21310 non-null  int64         
 9   lease_commence_date  21310 non-null  datetime64[ns]
 10  resale_price         21310 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(4)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 35642 entries, 0 to 167

In [21]:
from sklearn.ensemble import RandomForestRegressor

def buildRandForest(prediction,predictors):
    # Split prediction and predictors data
    y = pd.DataFrame(data[prediction])
    x = pd.DataFrame(data[predictors])
    x_trn,x_tst,y_trn,y_tst = train_test_split(x,y,test_size=0.25)
    
    print(prediction)
    print(x)
    print(y)
    
    # Train decision tree
    rf = RandomForestRegressor()
    rf.fit(x_trn, y_trn)
    
    # Predict Response corresponding to Predictors
    y_trn_pred = rf.predict(x_trn)
    y_tst_pred = rf.predict(x_tst)
    
    # Check the Goodness of Fit (on Train Data)
    print("Goodness of Fit of Model \tTrain Dataset")
    print("Explained variance\t:", rf.score(x_trn,y_trn))
    print("Mean squared error \t:", mean_squared_error(y_trn, y_trn_pred))
    print()

    # Check the Goodness of Fit (on Test Data)
    print("Goodness of Fit of Model \tTest Dataset")
    print("Explained variance \t:", rf.score(x_tst,y_tst))
    print("Mean squared error \t:", mean_squared_error(y_tst, y_tst_pred))
    print()
    
    # Plot (two) trees
#     for i in range(min(2,len(rf.estimators_))):
#         plt.figure(figsize=(30,15), dpi=300)
#         plot_tree(rf.estimators_[i])
#         plt.show()    
#==============================================================#

data.dropna(inplace=True)
buildRandForest('resale_price',['floor_area_sqm','lease_commence_date'])

resale_price
        floor_area_sqm lease_commence_date
0                   45          1986-01-01
1                   44          1980-01-01
2                   68          1980-01-01
3                   67          1984-01-01
4                   67          1980-01-01
...                ...                 ...
167393             131          1987-01-01
167394             122          1987-01-01
167395             122          1987-01-01
167396             146          1987-01-01
167397             146          1988-01-01

[167398 rows x 2 columns]
        resale_price
0           250000.0
1           265000.0
2           315000.0
3           320000.0
4           321000.0
...              ...
167393           0.0
167394           0.0
167395           0.0
167396           0.0
167397           0.0

[167398 rows x 1 columns]


TypeError: The DType <class 'numpy.dtype[int64]'> could not be promoted by <class 'numpy.dtype[datetime64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>)